In [3]:
import pandas as pd
import plotly.express as px
import joblib



In [ ]:
data = pd.read_csv('../data/pokemon_data_cleaned.csv')

df = data.copy()

df = pd.DataFrame(df)

df.head()

In [5]:
xgboost_model = joblib.load('../models/xgboost_model.pkl')
df['type'] = df['primary_type']

df_encoded = pd.get_dummies(df, columns=['type', 'pokemon_class'])

In [6]:
def create_battle_row(pokemon, opponent, attack_first):
    return pd.Series({
        'attack': pokemon['attack'],
        'defense': pokemon['defense'],
        'speed': pokemon['speed'],
        'hp': pokemon['hp'],
        'effectiveness': pokemon.get(f'{opponent["primary_type"].lower()}_attack_effectiveness', 1),
        'attack_first': attack_first,
        'pokemon_class_Legendary': pokemon['pokemon_class_Legendary'],
        'pokemon_class_Mythical': pokemon['pokemon_class_Mythical'],
        'pokemon_class_Normal': pokemon['pokemon_class_Normal']
    })
    
def simulate_battle(pokemon, opponent, model):
    attack_first_pokemon = int(pokemon['speed'] > opponent['speed'])
    attack_first_opponent = int(opponent['speed'] > pokemon['speed'])

    pokemon_row = create_battle_row(pokemon, opponent, attack_first_pokemon)
    opponent_row = create_battle_row(opponent, pokemon, attack_first_opponent)

    pokemon_wins = model.predict(pokemon_row.values.reshape(1, -1))[0] == 1
    opponent_wins = model.predict(opponent_row.values.reshape(1, -1))[0] == 1

    return {
        'pokemon_name': pokemon['name'],
        'opponent_name': opponent['name'],
        'pokemon_starts': bool(attack_first_pokemon),
        'pokemon_wins': pokemon_wins,
        'pokemon_attack': pokemon['attack'],
        'opponent_attack': opponent['attack'],
        'pokemon_defense': pokemon['defense'],
        'opponent_defense': opponent['defense'],
        'opponent_wins': opponent_wins
    }
    
def simulate_one_vs_all(pokemon, all_pokemon, model):

    battle_results = []
    seen_battles = set()
    battle_id = 1 

    for _, opponent in all_pokemon[all_pokemon['name'] != pokemon['name']].iterrows():
        battle_tuple = tuple(sorted([pokemon['pokedex_number'], opponent['pokedex_number']]))
        
        if battle_tuple not in seen_battles:
            seen_battles.add(battle_tuple)
            
            # Simulera striden
            battle_result = simulate_battle(pokemon, opponent, model)
            battle_result.update({
                'battle_id': battle_id,
                'opponent_name': opponent['name']
            })

            battle_results.append(battle_result)
            battle_id += 1

    return pd.DataFrame(battle_results)


pikachu = df_encoded[df_encoded['name'] == 'Pikachu'].iloc[0]
results_df = simulate_one_vs_all(pikachu, df_encoded, xgboost_model)
